Імпортуєм основні модулі

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

Імпортуєм дані

In [ ]:
data_frame = pd.read_csv("../input/fraud-rate/main_data.csv")

Перегляд таблиці даних

In [ ]:
data_frame.head()

Перегляд розмірів фрейму

In [ ]:
data_frame.shape

Перегляд параметрів транзакцій

In [ ]:
data_frame.columns

Розбиття фрейму(Х - параметри класифікації, Y - висновок класифікації)

In [ ]:
X = data_frame.drop(["PotentialFraud"], axis=1)
Y = data_frame["PotentialFraud"]

X_values = X.values
Y_values = Y.values

Розбиття на навчальну та тестову вибірки

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_values, 
                                                    Y_values, 
                                                    test_size = 0.25)

Побудова класифікатора з стандартними параметрами

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

RANDOM_FOREST = RandomForestClassifier()

Навчання класифікатора

In [ ]:
RANDOM_FOREST.fit(X_train, Y_train)

Передбачення класифікатора на тестовій вибірці

In [ ]:
Y_predict = RANDOM_FOREST.predict(X_test)

Оцінка мінімальної точності

In [ ]:
frauded_cards = data_frame[data_frame["PotentialFraud"] == 1]
safe_cards = data_frame[data_frame["PotentialFraud"] == 0]

part_of_frauded_cards = 100 * len(frauded_cards)/len(data_frame)
min_accurancy = 100 - part_of_frauded_cards
round(min_accurancy, 3)

Оцінка точності класифікатора без налаштуквання

In [ ]:
from sklearn.metrics import roc_auc_score

r_accuracy = roc_auc_score(Y_predict, Y_test)
round(r_accuracy, 3)

Імпорт моделі пошуку найкращих параметрів

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Задання потенційно найкращих параметрів

In [ ]:
n_estiamtors = [5, 10, 30, 50, 70, 90, 110, 130, 150, 170]
max_features = ["auto", "sqrt", "log2"]
max_depth = [5, 7, 8, 9, 10, 11, 12]
min_samples_split = [2, 5, 10, 15, 20]
min_samples_leaf = [1, 2, 5, 10, 15]

Створення сітки параметрів

In [ ]:
grid_param = {
    "n_estimators" : n_estiamtors,
    "max_features" : max_features,
    "max_depth" : max_depth,
    "min_samples_split" : min_samples_split,
    "min_samples_leaf" : min_samples_leaf
}

Пошук параметрів

In [ ]:
rd = RandomForestClassifier()
rdr = RandomizedSearchCV(estimator = rd, param_distributions = grid_param, n_iter = 100, cv = 5, verbose = 2, n_jobs = -1)
rdr.fit(X_train, Y_train)

Вивід результату

In [ ]:
rdr.best_params_

Звуження проміжків

In [ ]:
n_estiamtors = [145, 150, 155, 160]
max_depth = [11, 12, 13]
min_samples_split = [1, 2, 3]

Сітка наступної ітерації

In [ ]:
grid_param = {
    "n_estimators" : n_estiamtors,
    "max_depth" : max_depth,
    "min_samples_split" : min_samples_split
}

Наступна ітерація пошуку

In [ ]:
rd = RandomForestClassifier()
rdr = RandomizedSearchCV(estimator = rd, param_distributions = grid_param, n_iter = 100, cv = 5, verbose = 2, n_jobs = -1)
rdr.fit(X_train, Y_train)

In [ ]:
rdr.best_params_

In [ ]:
n_estiamtors = [152, 154, 155, 156, 157]
min_samples_split = [3, 4, 5]

In [ ]:
grid_param = {
    "n_estimators" : n_estiamtors,
    "min_samples_split" : min_samples_split
}

In [ ]:
rd = RandomForestClassifier(max_depth = 12, min_samples_leaf = 1, max_features = 'auto')
rdr = RandomizedSearchCV(estimator = rd, param_distributions = grid_param, n_iter = 100, cv = 5, verbose = 2, n_jobs = -1)
rdr.fit(X_train, Y_train)

In [ ]:
rdr.best_params_

In [ ]:
n_estiamtors = [155, 156, 157, 158]

In [ ]:
grid_param = {
    "n_estimators" : n_estiamtors
}

In [ ]:
rd = RandomForestClassifier(max_depth = 12, min_samples_leaf = 1, max_features = 'auto', min_samples_split = 4)
rdr = RandomizedSearchCV(estimator = rd, param_distributions = grid_param, n_iter = 100, cv = 5, verbose = 2, n_jobs = -1)
rdr.fit(X_train, Y_train)

In [ ]:
rdr.best_params_

In [ ]:
Побудуємо класифікатор із знайденими параметрами

In [ ]:
RANDOM_FOREST_WITH_BEST_PARAM = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='log2',
                       max_leaf_nodes=60, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=85,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Проведемо його навчання

In [ ]:
RANDOM_FOREST_WITH_BEST_PARAM.fit(X_train, Y_train)

Зробимо передбачення

In [ ]:
Y_predict = RANDOM_FOREST_WITH_BEST_PARAM.predict(X_test)

Оцінимо точність

In [ ]:
round(roc_auc_score(Y_predict, Y_test), 3)

Збалансуєм базу

In [ ]:
frauded_cards_oversampled = frauded_cards.sample(len(safe_cards), replace = True)

oversampled_data = pd.concat([frauded_cards_oversampled, safe_cards], axis = 0)

Зробимо вибірки

In [ ]:
X_values_o = oversampled_data.drop("PotentialFraud", axis = 1).values
Y_values_o = oversampled_data.PotentialFraud.values

X_train_o, X_test_o, Y_train_o, Y_test_o = train_test_split(X_values_o, Y_values_o, test_size = .25)

Оцінимо точність класифікатора без підбору параметрів на збалансованій базі

In [ ]:
RANDOM_FOREST_OVERSAMPLING = RandomForestClassifier()
RANDOM_FOREST_OVERSAMPLING.fit(X_train_o, Y_train_o)
Y_o_predict = RANDOM_FOREST_OVERSAMPLING.predict(X_test_o)
roc_auc_score(Y_o_predict, Y_test_o)
cross_val_score(RANDOM_FOREST_OVERSAMPLING, X_train_o, Y_train_o, cv=5).mean()

In [ ]:
n_estiamtors = [5, 10, 30, 50, 70, 90, 110, 130, 150, 170]
max_features = ["auto", "sqrt", "log2"]
max_depth = [5, 7, 8, 9, 10, 11, 12]
min_samples_split = [2, 5, 10, 15, 20]
min_samples_leaf = [1, 2, 5, 10, 15]

In [ ]:
grid_param = {
    "n_estimators" : n_estiamtors,
    "max_features" : max_features,
    "max_depth" : max_depth,
    "min_samples_split" : min_samples_split,
    "min_samples_leaf" : min_samples_leaf
}

In [ ]:
rd = RandomForestClassifier()
rdr = RandomizedSearchCV(estimator = rd, param_distributions = grid_param, n_iter = 100, cv = 5, verbose = 2, n_jobs = -1)
rdr.fit(X_train_o, Y_train_o)

In [ ]:
rdr.best_params_

In [ ]:
n_estiamtors = [120, 125, 130, 135, 140]
max_depth = [11, 12, 13]
min_samples_split = [3, 4, 5, 6, 7]

In [ ]:
grid_param = {
    "n_estimators" : n_estiamtors,
    "max_depth" : max_depth,
    "min_samples_split" : min_samples_split
}

In [ ]:
rd = RandomForestClassifier(min_samples_leaf = 1, max_features = "sqrt")
rdr = RandomizedSearchCV(estimator = rd, param_distributions = grid_param, n_iter = 100, cv = 5, verbose = 2, n_jobs = -1)
rdr.fit(X_train_o, Y_train_o)

In [ ]:
rdr.best_params_

In [ ]:
n_estiamtors = [118, 119, 120, 121, 120]

In [ ]:
grid_param = {
    "n_estimators" : n_estiamtors
}

In [ ]:
rd = RandomForestClassifier(min_samples_leaf = 1, max_features = "sqrt", min_samples_split = 4, max_depth = 12)
rdr = RandomizedSearchCV(estimator = rd, param_distributions = grid_param, n_iter = 100, cv = 5, verbose = 2, n_jobs = -1)
rdr.fit(X_train_o, Y_train_o)

In [ ]:
rdr.best_params_

In [ ]:
RANDOM_FOREST_OVERSAMPLING_WITH_BEST_PARAMS = RandomForestClassifier(n_estimators = 120,
                                                                    max_depth = 12, 
                                                                    min_samples_leaf = 1,
                                                                    max_features = "sqrt",
                                                                    min_samples_split = 4)

Модель із знайденими параметрами

In [ ]:
RANDOM_FOREST_OVERSAMPLING_WITH_BEST_PARAMS.fit(X_train_o, Y_train_o)

Навчання моделі

In [ ]:
Y_predict_o = RANDOM_FOREST_OVERSAMPLING_WITH_BEST_PARAMS.predict(X_test_o)

Оціним точність

In [ ]:
round(roc_auc_score(Y_predict_o, Y_test_o), 2)

In [ ]:
cross_val_score(RANDOM_FOREST_OVERSAMPLING_WITH_BEST_PARAMS, X_train_o, Y_train_o, cv=5).mean()